# Anime Mini Deep Learning Model
This notebook trains a tiny CNN on synthetic anime-style image data (dummy data) and demonstrates saving the model.
At the end, Streamlit deployment instructions and example code are provided.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Dummy dataset of 100 'anime' images sized 3x32x32
X = torch.randn(100, 3, 32, 32)
y = torch.randint(0, 2, (100,))  # Binary classification

class SmallAnimeCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32*8*8, 2)

    def forward(self,x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

model = SmallAnimeCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (very short)
for epoch in range(2):
    optimizer.zero_grad()
    preds = model(X)
    loss = criterion(preds, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

torch.save(model.state_dict(), 'anime_model.pth')
print('Model saved as anime_model.pth')

## Streamlit Deployment Instructions

Create a file `app.py` with the following:
```python
import streamlit as st
import torch
from model import SmallAnimeCNN
from PIL import Image
import torchvision.transforms as T

st.title("Anime Mini Model Demo")
model = SmallAnimeCNN()
model.load_state_dict(torch.load('anime_model.pth', map_location='cpu'))
model.eval()

uploaded = st.file_uploader("Upload anime image", type=['png','jpg','jpeg'])
if uploaded:
    img = Image.open(uploaded).convert('RGB')
    st.image(img)
    transform = T.Compose([T.Resize((32,32)), T.ToTensor()])
    x = transform(img).unsqueeze(0)
    preds = model(x)
    label = torch.argmax(preds,1).item()
    st.write(f"Prediction: {label}")
```

Then deploy this to Streamlit Cloud. Upload all files including `anime_model.pth`, `model.py` (containing the class), and `app.py`.